### Ejercicios del tema de Ensemble Learning y Random Forests

*Hugo Díaz Díaz* (*hdiazd00@estudiantes.unileon.es*)

*Correo profesional: hugo.didi.contacto@gmail.com*

---


## Parte teórica (opcional)

### 1. Si has entrenado cinco modelos diferentes con exactamente los mismos datos de entrenamiento, y todos logran una precisión del 95%, ¿hay alguna posibilidad de combinar estos modelos para obtener mejores resultados? Si es así, ¿cómo? Si no, ¿por qué?

Sí, se pueden combinar.
Si los cinco modelos no cometen exactamente los mismos errores, es decir, son lo suficientemente diversos en cuanto a algoritmo, hiperparámetros inicializaciones... un ensemble (por ejemplo, un clasificador de votación dura o suave) puede mejorar un poco la precisión respecto a cada modelo individual, porque promediar las predicciones reduce la varianza del sistema. Si fueran prácticamente idénticos, la mejora sería nula.

### 2. ¿Cuál es la diferencia entre clasificadores de voto duro y voto suave?

En voto duro cada modelo da una clase y la salida final es la clase más votada. En voto suave, cada modelo da probabilidades por clase, se promedian esas probabilidades y se elige la de mayor probabilidad media. El voto suave suele ir mejor porque tiene en cuenta el grado de confianza de los modelos.

### 3. ¿Es posible acelerar el entrenamiento de un ensemble de bagging distribuyéndolo entre múltiples servidores? ¿Qué pasa con los ensembles de pasting, boosting, random forests o stacking?

En bagging y pasting cada modelo se entrena de forma independiente en un subconjunto de datos, así que se pueden repartir entre varios núcleos/servidores sin problema. Lo mismo vale para los Random Forests, que son básicamente bagging de árboles. En boosting (AdaBoost, Gradient Boosting) los modelos se entrenan secuencialmente, cada uno corrige al anterior, así que no se puede paralelizar la secuencia. En stacking, los modelos base sí se pueden entrenar en paralelo, pero el meta-modelo va después con sus salidas.

### 4. ¿Cuál es el beneficio de la evaluación out-of-bag?

En bagging, cada modelo solo ve alrededor del 63 % de las instancias de entrenamiento, el resto se consideran out-of-bag para ese modelo. Eso permite usarlas como “test interno” sin tener que reservar un conjunto de validación separado, aprovechando todos los datos tanto para entrenar como para estimar el rendimiento del ensemble.

### 5. ¿Qué hace que los Extra-Trees sean más aleatorios que los Random Forests regulares? ¿Cómo puede ayudar esta aleatoriedad extra? ¿Son los Extra-Trees más lentos o más rápidos que los Random Forests regulares?

Los Extra-Trees son más aleatorios porque, además de muestrear características como en un Random Forest, eligen los umbrales de corte de forma totalmente aleatoria en cada nodo, en vez de buscar el mejor split. Esta aleatoriedad extra reduce la correlación entre árboles, baja la varianza (aunque suba un poco el sesgo) y suele mejorar la generalización. Al no buscar el mejor umbral, suelen ser más rápidos de entrenar que un Random Forest estándar.

### 6. Si tu ensemble de AdaBoost subajusta los datos de entrenamiento, ¿qué hiperparámetros deberías ajustar y cómo?

Si mi ensemble de **AdaBoost** está subajustando (no llega a aprender bien ni siquiera el conjunto de entrenamiento), lo que necesito es darle más capacidad al modelo. Desde el punto de vista de los hiperparámetros, las opciones típicas son:

- Aumentar `n_estimators`, es decir, añadir más clasificadores débiles al ensemble para que tenga más pasos de corrección de errores. 

- Hacer el estimador base algo más complejo (por ejemplo, árboles con mayor `max_depth` en lugar de simples decision stumps).

- Y, si sigo muy corto, se puede subir ligeramente `learning_rate`, porque una tasa de aprendizaje muy baja actúa como una regularización fuerte y puede estar frenando demasiado las correcciones.

### 7. Si tu ensemble de Gradient Boosting sobreajusta el conjunto de entrenamiento, ¿deberías aumentar o disminuir la tasa de aprendizaje?

En **Gradient Boosting**, el hiperparámetro `learning_rate` controla cuánto corrige cada nuevo árbol los errores de los anteriores. Un valor alto hace que cada árbol “empuje” mucho y es fácil que el modelo sobreajuste al conjunto de entrenamiento. Cuando veo overfitting, lo que debo hacer es disminuir la tasa de aprendizaje y, en compensación, permitir más árboles (`n_estimators` más grande). En las transparencias se comenta precisamente que valores bajos de `learning_rate` requieren más árboles pero tienden a generalizar mejor gracias a esta estrategia de *shrinkage*.

---

## Parte práctica (obligatoria)

### Ejercicio 8

Carga los datos MNIST (introducidos en el Capítulo 3) y divídelos en un conjunto de entrenamiento, un conjunto de validación y un conjunto de prueba (por ejemplo, usa 50,000 instancias para entrenamiento, 10,000 para validación y 10,000 para prueba).

- Luego entrena varios clasificadores, como un clasificador Random Forest, un clasificador Extra-Trees y un SVM.
- A continuación, intenta combinarlos en un ensemble que supere a todos en el conjunto de validación, utilizando un clasificador de voto suave o duro.
- Una vez que hayas encontrado uno, pruébalo en el conjunto de prueba.
- ¿Cuánto mejor se desempeña en comparación con los clasificadores individuales?

Primero se cargan los datos de MNIST desde `fetch_openml`. Después se normalizan las imágenes y se realiza la partición en conjuntos de entrenamiento, validación y prueba con el esquema 50k/10k/10k.

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

import numpy as np

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
import os
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "ensembles"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

from sklearn.datasets import fetch_openml
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

np.random.seed(42)

In [9]:
mnist = fetch_openml('mnist_784', version=1, as_frame=False)

X = mnist["data"]
y = mnist["target"].astype(np.uint8)  # etiquetas como enteros

# Normalización sencilla [0, 1]
X = X / 255.0

# División 60k / 10k (como en el libro) y luego 50k / 10k / 10k
X_train_full, X_test = X[:60000], X[60000:]
y_train_full, y_test = y[:60000], y[60000:]

X_train, X_val = X_train_full[:50000], X_train_full[50000:]
y_train, y_val = y_train_full[:50000], y_train_full[50000:]

X_train.shape, X_val.shape, X_test.shape


((50000, 784), (10000, 784), (10000, 784))